In [ ]:
from ultralytics import YOLO

model = YOLO('./best.pt')
model.export(format='tflite')

In [ ]:
from ultralytics import YOLO
import onnx2tf

model = YOLO('./best.pt')
model.export(format='onnx',opset=15)

### from ONNX to TF

In [ ]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load("./best.onnx")

# Convert ONNX model to TensorFlow
tf_rep = prepare(onnx_model)

# Export the TensorFlow model to a temporary directory
tf_rep.export_graph("./temp")

default settings without optimizations

In [ ]:
import tensorflow as tf

tf_model_path_directory = './temp/'
tflite_model_path = './tf_lite_model.tflite'

# Convert the TensorFlow model to TFLite with TF Select ops enabled
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path_directory)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]

tflite_model = converter.convert()

# Save the TFLite model
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"ONNX model has been successfully converted to TFLite and saved at {tflite_model_path}")

### Optimized tf_lite version

In [ ]:
import tensorflow as tf

tf_model_path = './temp/'
tflite_model_path = './tf_lite_model_optimized.tflite'
# Convert the TensorFlow model to TFLite with TF Select ops enabled
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]

# Apply optimization for quantization
"there are different optimizations available, we're just using the default optimization"
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save the TFLite model
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"ONNX model has been successfully converted to TFLite and saved at {tflite_model_path}")

### Inference for quantized model

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="tf_lite_model_optimized.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

In [ ]:
# Prepare input data
input_data = np.random.randn(1,3,288,288).astype(np.float32)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Process output data
print(output_data)

In [ ]:
print(output_data.shape)